In [1]:
import os

import optuna
from Recommenders.DataIO import DataIO
from Evaluation.Evaluator import EvaluatorHoldout

In [2]:
from utils import load_data2, split_data2

data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)

In [3]:
from utils import get_database_url

study_name = "SLIMElastic"
study = optuna.create_study(
    study_name=study_name,
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2023-12-17 11:34:50,823] Using an existing study with name 'SLIMElastic' instead of creating a new one.


In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from utils import evaluator

def objective(trial):
    alpha = trial.suggest_float('alpha', 1e-5, 1, log=True)
    topK = trial.suggest_int('topK', 1, 1000, log=True)
    l1_ratio = trial.suggest_float('l1_ratio', 1e-8, 1, log=True)
    
    recommender = MultiThreadSLIM_SLIMElasticNetRecommender(data_train, verbose=False)
    recommender.fit(alpha=alpha, l1_ratio=l1_ratio, topK=topK)
    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)
    
    return ev_map

study.optimize(objective, n_trials=75)

In [4]:
study.best_params

{'alpha': 0.003495947934879827, 'topK': 531, 'l1_ratio': 0.0030148508221821105}

In [5]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender

recommender = MultiThreadSLIM_SLIMElasticNetRecommender(data_train, verbose=False)
recommender.fit(**study.best_params)

In [10]:
from utils import evaluator
pr, re, ev_map, ev_ndcg, ev_auc = evaluator(recommender, data_train, data_val)
print("Precision: {:.8f}".format(pr))
print("Recall: {:.8f}".format(re))
print("MAP: {:.8f}".format(ev_map))

Precision: 0.09414074
Recall: 0.13879554
MAP: 0.08188953


In [6]:
from utils import submission2

submission2(recommender, users, usermap, itemmap, data_train)

TopPopRecommender: URM Detected 233 ( 1.8%) users with no interactions.
TopPopRecommender: URM Detected 137 ( 0.6%) items with no interactions.
